# Analysing Piwik Logs

We will stream a JSON array of objects as we proceed into a LZMH compressed file.

In [1]:
var glob = require('glob');
var path = require('path');
var fs = require('fs');
var JSONStream = require('JSONStream')
var EventStream = require('event-stream');
var xz = require('xz');

# Data Selection

We select all timestamped files — we have one stat file per day.

In [2]:
var source_dir = 'raw_visit_logs/';
var target_dir = 'deduplicated_logs/';
var files = glob.sync(source_dir + '*-*-*.json');

Some utility functions.

In [3]:
var dateFromTimestamp = (ts) => new Date(ts * 1000).toISOString().split('T')[0];

In [8]:
var all = files.slice(0, 1).map(file => {
    const writer = JSONStream.stringify()
        .pipe(fs.createWriteStream(target_dir + path.basename(file)));
    
    return fs.createReadStream(file)
        .pipe(JSONStream.parse('*'))
        .pipe(EventStream.mapSync(({ idVisit, actionDetails, serverDate }) => {
            var actions = actionDetails
                .map(({ url, type, siteSearchKeyword:keyword, timestamp }) => ({
                  url,
                  type,
                  keyword,
                }))
                .filter(({type, keyword}, i, allActions) => {
                    const nextKeyword = allActions[i+1] ? allActions[i+1].keyword : null;

                    return !(type === 'search' && nextKeyword && nextKeyword.indexOf(keyword) === 0);
                });

            var visitClean = {
                idVisit,
                actions,
                date: serverDate,
            };
        

            return new Buffer(visitClean);
        }))
        .pipe(writer)
        .on('end', () => writer.end());
});





Async Error: First argument must be a string, Buffer, ArrayBuffer, Array, or array-like object.